# CONTROLLO MATCHING

In [20]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased, Cupid
import pprint
import numpy as np

def list2string(x):
    result = ''
    if(isinstance(x, list)):
        for element in x:
            result = result + ', ' + str(element)
        return result[1:]
    else:
        return x

# Load data using pandas
df1 = pd.read_json("schema_mediato_confronto.json")
# df2 = pd.read_json("datasets/output_globaldata.json")
df2 = pd.read_json("datasets/valueToday_dataset.jsonl", lines=True)
# df2 = df2.apply(np.vectorize(list2string))
df2 = df2.applymap(list2string)
display(df2)
# Instantiate matcher and run
# Coma requires java to be installed on your machine
# If java is not an option, all the other algorithms are in Python (e.g., Cupid)
# matcher = Coma(strategy="COMA_OPT")
matcher = DistributionBased(0.9, 0.9)
matches = valentine_match(df1, df2, matcher)
# print(type(matches))
# print(matches)
pp = pprint.PrettyPrinter(indent=4)

,id,name,world_rank,annual_revenue_in_usd,annual_net_income_in_usd,annual_results_for_year_ending,total_assets_in_usd,total_liabilities_in_usd,total_equity_in_usd,headquarters_region_city,headquarters_country,headquarters_sub_region,headquarters_continent,company_business,number_of_employees,ceo,founders,company_website
0,b9a098214d1a4ce1acc834cf17aef660,Apple,1,"394,330 Million USD","99,803 Million USD",Sep-2022,"352,760 Million USD","302,080 Million USD","50,672 Million USD",California,USA,Northern America,Americas,"Technology, Mobiles & Accessories, Electronic...","154,000",Tim Cook,"Steve Jobs, Steve Wozniak, Ronald Wayne",https://www.apple.com/
1,42f1b7d5a40343de9837fc75d74767c3,Taiwan Semiconductor Manufacturing Company (tsmc),10,"55,306 Million USD","20,783 Million USD",Dec-2021,"129,800 Million USD","54,253 Million USD","75,543 Million USD",Hsinchu,Taiwan,South East Asia,Asia,"Technology, Semiconductors, Electronics, Tech...","65,152",C. C. Wei,Morris Chang,https://www.tsmc.com/
2,85fa8c7dfbb447d898002cb3cfb2fffd,Nvidia Corporation,9,"26,914 Million USD","9,752 Million USD",For Year Ending JAN-2022,"44,187 Million USD","17,575 Million USD","26,612 Million USD",California,USA,Northern America,Americas,"Technology, Semiconductors, Electronics, Tech...","18,975",Jensen Huang,"Jensen Huang, Chris Malachowsky, Curtis Priem",http://www.nvidia.com/
3,6b52bb9b50924235b1003c74bb8db227,Berkshire Hathaway,8,"354,640 Million USD","89,795 Million USD",For Year ending Dec-2021,"956,780 Million USD","443,850 Million USD","514,930 Million USD",Nebraska,USA,Northern America,Americas,"Financial Services, Investments, Insurance","372,000",Warren Buffett,Oliver Chace,https://www.berkshirehathaway.com/
4,d10c3bd4fe924f138454b6f0583de7a1,Meta Platforms,7,"117,930 Million USD","39,370 Million USD",Dec-2021,"165,990 Million USD","41,108 Million USD","124,880 Million USD",California,USA,Northern America,Americas,"Communication Services, Internet or Mobile Ap...","77,805",Mark Zuckerberg,"Mark Zuckerberg, Andrew McCollum, Eduardo Sav...",https://www.facebook.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10677,b08a3d47642d44e7a10f8582a14cf9f9,Nilfisk Holding A/s,"9,992",994 Million USD,51 Million USD,For Year Ending Dec-2021,841 Million USD,633 Million USD,207 Million USD,NaN,Denmark,Northern Europe,Europe,"Industries, Industry Machinery",NaN,NaN,NaN,https://www.nilfisk.com/
10678,6622d097ddd74d8c8c1be4cec195dfb6,Taikisha,"9,991","1,527 Million USD",53 Million USD,For Year Ending Mar-2022,"1,665 Million USD",711 Million USD,954 Million USD,NaN,Japan,Eastern Asia,Asia,"Industries, Electronics, Paints and Coatings,...","5,042",NaN,NaN,https://www.taikisha-group.com/
10679,42e3f369762b44d7977fda3584145175,Parque Arauco,"9,990",189 Million USD,26 Million USD,For Year Ending Dec-2021,"3,084 Million USD","1,651 Million USD","1,433 Million USD",NaN,Chile,South America,Americas,Real Estate,NaN,NaN,NaN,https://www.parauco.com/
10680,dda34d322ea9415aa399a26e207e6a64,Talos Energy,"9,999","1,244 Million USD",-182 Million USD,Year ending Dec-2021,"2,766 Million USD","2,006 Million USD",760 Million USD,Texas,USA,Northern America,Americas,"Energy, Oil and Gas",443,NaN,NaN,https://www.talosenergy.com


In [3]:
pp.pprint(matches)

{   (('table_1', 'area_served'), ('table_2', 'headquarters_continent')): 0.7939191705522204,
    (('table_1', 'country'), ('table_2', 'headquarters_country')): 0.8697265607735573,
    (('table_1', 'employees'), ('table_2', 'number_of_employees')): 0.9268644538778984,
    (('table_1', 'employees'), ('table_2', 'world_rank')): 0.8685681715851138,
    (('table_1', 'headquarters'), ('table_2', 'headquarters_region_city')): 0.9462259059955659,
    (('table_1', 'name'), ('table_2', 'name')): 0.8835215044000037,
    (('table_1', 'sector'), ('table_2', 'company_business')): 0.824344321652341,
    (('table_1', 'website'), ('table_2', 'company_website')): 0.9141177743412015}


# CREAZIONE SCHEMA MEDIATO

In [33]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased
import pprint

In [34]:
# ---------------------------------------------------------
# CREATA PER INIZIALIZZARE UN DATASET FITTIZIO MA ORA NON
# SI UTILIZZA IL DATASET FITTIZIO DA NOI CREATO MANUALMENTE
# ---------------------------------------------------------
def creation_dataset_confronto(df1):
  # Load data using pandas
  df2 = pd.read_json("datasets/output_disfold.json")

  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])

  for i in df2.index:
      data = {}
      # print(i)
      for d,m in zip(attributi_dataset,attributi_mediato):
        # print(df2[d][i])
        data[m] = df2[d][i]
      temp_data=pd.DataFrame([data])
      df1=pd.concat([df1, temp_data], ignore_index=True)

  return df1


In [35]:
def creation_df_log_matcher(df_logmachers, matches, i, type, name_dataset):
  record = {}
  record['tab1'] = i[0][0]
  record['attr1'] = i[0][1]
  record['tab2'] = name_dataset
  record['attr2'] = i[1][1]
  record['prob'] = matches[i]
  if type == 'coma':
    record['matcher'] = 'Coma'
  else:
    record['matcher'] = 'DistributionBased'
  temp_record=pd.DataFrame([record])
  # df_logmachers=df_logmachers.append(record, ignore_index=True)
  df_logmachers=pd.concat([df_logmachers, temp_record], ignore_index=True)
  return df_logmachers

In [36]:
def creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset):

  # Instantiate matcher and run
  # Coma requires java to be installed on your machine
  # If java is not an option, all the other algorithms are in Python (e.g., Cupid)
  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  distr_run = False

 
  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    df_logmachers = creation_df_log_matcher(df_logmachers, matches_coma, i, 'coma', name_dataset)
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])
    else:
      try:
        if not distr_run:
          matcher_distr = DistributionBased()
          matches_distr = valentine_match(df_confronto, df2, matcher_distr)
          distr_run = True
        # distribution based già instanziato
        # log del matcher su Distribution Based
        df_logmachers = creation_df_log_matcher(df_logmachers, matches_distr, i, 'distribution', name_dataset)
        # se nel Distribution Based c'è la stessa coppia di attributi 
        # che fa match con valore superiore a 0.5 allora fa match
        if matches_distr[i] > 0.5:
          attributi_mediato.append(i[0][1])
          attributi_dataset.append(i[1][1])
      except:
        pass
  # print(attributi_dataset)
  # print(attributi_mediato)

  # inserimento dei dati nel dataframe
  for i in df2.index:
    data = {}
    # print(i)
    for d,m in zip(attributi_dataset,attributi_mediato):
      # print(df2[d][i])
      data[m] = df2[d][i]
    temp_data=pd.DataFrame([data])
    df1=pd.concat([df1, temp_data], ignore_index=True)
    df_confronto=pd.concat([df_confronto, temp_data], ignore_index=True)

  pp = pprint.PrettyPrinter(indent=4)

  # stampa dei matchers
  pp.pprint(matches_coma)
  if distr_run:
    pp.pprint(matches_distr)

  return df1, df_logmachers
  # print(df1)


In [37]:
#funzione utilizzata per trasformare gli elementi del dataframe che sono una lista in stringa 
def list2string(x):
    result = ''
    if(isinstance(x, list)):
        for element in x:
            result = result + ', ' + str(element)
        return result[1:]
    else:
        return x

In [38]:
from pathlib import Path
#formati json, csv, xls, xlsx, jsonl
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'founders', 'share_price', 'website', 'stock', 'area_served', 'earnings'])
# df_confronto = creation_dataset_confronto(df1)
df_confronto = pd.read_json('schema_mediato_auto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])

#numero di errori nell'esecuzione di file
num_error = 0

directory = 'datasets'
dataset_list_json = Path(directory).glob('*.json')
dataset_list_jsonl = Path(directory).glob('*.jsonl')
dataset_list_csv = Path(directory).glob('*.csv')
dataset_list_xls = Path(directory).glob('*.xls')
dataset_list_xlsx = Path(directory).glob('*.xlsx')
i = 0

# --------------------------------------------------------------------
# Modificare seleziona_tipo per scegliere che tipo di file eseguire
# --------------------------------------------------------------------
seleziona_tipo = 0

# CON SELEZIONA_TIPO=0 SI RUNNANO I TUTTI TIPI DI FILE
# CON SELEZIONA_TIPO=1 SI RUNNANO I JSON
if seleziona_tipo==0 or seleziona_tipo==1:
  for path in dataset_list_json:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path)
    # trasforma le celle del dataframe che hanno liste in stringhe
    df2 = df2.applymap(list2string)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=2 SI RUNNANO I JSONL
if seleziona_tipo==0 or seleziona_tipo==2:
  for path in dataset_list_jsonl:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path, lines=True)
    # trasforma le celle del dataframe che hanno liste in stringhe
    df2 = df2.applymap(list2string)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except Exception as e:
      print(e)
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=3 SI RUNNANO I CSV
if seleziona_tipo==0 or seleziona_tipo==3:
  for path in dataset_list_csv:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_csv(path)
    # trasforma le celle del dataframe che hanno liste in stringhe
    df2 = df2.applymap(list2string)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=4 SI RUNNANO I XLS
if seleziona_tipo==0 or seleziona_tipo==4:
  for path in dataset_list_xls:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    # trasforma le celle del dataframe che hanno liste in stringhe
    df2 = df2.applymap(list2string)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=5 SI RUNNANO I XLSX
if seleziona_tipo==0 or seleziona_tipo==5:
  for path in dataset_list_xlsx:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    # trasforma le celle del dataframe che hanno liste in stringhe
    df2 = df2.applymap(list2string)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


print("Numero errori: " + str(num_error))
display(df1)
print('--------DATAFRAME LOG MATCHERS--------')
display(df_logmachers)

0 datasets\cbinsights.json
{   (('table_1', 'country'), ('table_2', 'country')): 0.78209436,
    (('table_1', 'founded'), ('table_2', 'datejoined')): 0.2843201,
    (('table_1', 'industry'), ('table_2', 'industry')): 0.7845453,
    (('table_1', 'name'), ('table_2', 'valuation')): 0.22594458,
    (('table_1', 'sector'), ('table_2', 'selectinvestors')): 0.35263473}
{   (('table_1', 'country'), ('table_2', 'country')): 0.9570572901040193,
    (('table_1', 'headquarters'), ('table_2', 'country')): 0.8676776244121108,
    (('table_1', 'stock'), ('table_2', 'selectinvestors')): 0.9394155946854038}
1 datasets\companiesmarketcap.json
{   (('table_1', 'country'), ('table_2', 'country')): 0.8030505,
    (('table_1', 'employees'), ('table_2', 'categories')): 0.2866888,
    (('table_1', 'market_cap'), ('table_2', 'marketcap')): 0.6679597,
    (('table_1', 'name'), ('table_2', 'name')): 0.79359674,
    (('table_1', 'share_price'), ('table_2', 'sharePrice')): 0.68009496,
    (('table_1', 'stock'), (

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 32473: invalid continuation byte

In [40]:
#formati csv
df1.to_csv('output_alignment/schema_mediato.csv')
df_logmachers.to_csv('output_alignment/log_matchers.csv')
df_confronto.to_csv('output_alignment/schema_mediato_incrementale_confronto.csv')
#formati json
df1.to_json('output_alignment/schema_mediato.json')
df_logmachers.to_json('output_alignment/log_matchers.json')
df_confronto.to_json('output_alignment/schema_mediato_incrementale_confronto.json')

# Il codice sottostante è utilizzato per generare il dataframe di un singolo dataset

In [31]:
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'founders', 'share_price', 'website', 'stock', 'area_served', 'revenue'])
df_confronto = pd.read_json('schema_mediato_auto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])


# df2 = pd.read_json('datasets/output_globaldata.json')
df2 = pd.read_json('datasets/output_wiki.json')
# trasforma le celle del dataframe che hanno liste in stringhe
df2 = df2.applymap(list2string)
df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, 'companiesMarketCap_dataset.jsonl')

display(df1)
display(df_logmachers)

{   (('table_1', 'employees'), ('table_2', 'number_of_employees')): 0.58523935,
    (('table_1', 'founded'), ('table_2', 'founded')): 0.8536217,
    (('table_1', 'founders'), ('table_2', 'founders')): 0.85607266,
    (('table_1', 'headquarters'), ('table_2', 'headquarters')): 0.8636485,
    (('table_1', 'industry'), ('table_2', 'industry')): 0.85607266,
    (('table_1', 'name'), ('table_2', 'name')): 0.8441679,
    (('table_1', 'revenue'), ('table_2', 'revenue')): 0.8536217,
    (('table_1', 'sector'), ('table_2', 'services')): 0.3683276,
    (('table_1', 'website'), ('table_2', 'website')): 0.8536217}
{   (('table_1', 'founded'), ('table_2', 'founded')): 0.828646947780304,
    (('table_1', 'name'), ('table_2', 'name')): 0.9849741062436634,
    (('table_1', 'revenue'), ('table_2', 'key_people')): 1.0,
    (('table_1', 'revenue'), ('table_2', 'number_of_employees')): 0.8718827771852302,
    (('table_1', 'revenue'), ('table_2', 'services')): 0.905237322683972,
    (('table_1', 'revenue')

,name,industry,market_cap,employees,country,headquarters,address,sector,ceo,founded,founders,share_price,website,stock,area_served,revenue
0,Binance,cryptocurrency industry,NaN,,NaN,,NaN,NaN,NaN,July 2017,"(""CZ""), Yi He",NaN,"www, .binance, .com",NaN,NaN,US$
1,Gourmet Master Co. Ltd,Food and Beverage,NaN,,NaN,"Taipei, Taiwan",NaN,NaN,NaN,2004,,NaN,"www.85cafe.com, [1], www.85cbakerycafe.com",NaN,NaN,NT$
2,Tata Consultancy Services,"Information technology, Consulting, Outsourcing",NaN,"616,171, [3], (October 2022)",NaN,"Mumbai, Maharashtra",NaN,NaN,NaN,1968,J. R. D. Tata,NaN,"www, .tcs, .com",NaN,NaN,", (US$25 billion), (2022)"
3,"APMEX, Inc.",precious metals,NaN,,NaN,"Oklahoma City, Oklahoma, United States",NaN,NaN,NaN,2000,,NaN,www.apmex.com,NaN,NaN,
4,Bitmain Technologies Ltd.,Cryptocurrency,NaN,,NaN,"Beijing, China",NaN,NaN,NaN,2013,"Micree Zhan, Jihan Wu",NaN,"bitmain, .com",NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,Blockstream,"Cryptocurrency, software",NaN,"50–100, [, citation needed, ], (2014)",NaN,"Victoria, British Columbia",NaN,NaN,NaN,2014,"Gregory Maxwell, Pieter Wuille, Matt Corallo,...",NaN,"blockstream, .com",NaN,NaN,
998,Nuri,Financial Services,NaN,,NaN,"Berlin, Germany",NaN,NaN,NaN,2015,,NaN,,NaN,NaN,
999,Bitstamp,,NaN,,NaN,,NaN,NaN,NaN,,,NaN,"www, .bitstamp, .net",NaN,NaN,
1000,BitPay,Payment Processor,NaN,,NaN,"Atlanta, Georgia, USA",NaN,NaN,NaN,,,NaN,,NaN,NaN,


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,headquarters,companiesMarketCap_dataset.jsonl,headquarters,0.863649,Coma
1,table_1,industry,companiesMarketCap_dataset.jsonl,industry,0.856073,Coma
2,table_1,founders,companiesMarketCap_dataset.jsonl,founders,0.856073,Coma
3,table_1,founded,companiesMarketCap_dataset.jsonl,founded,0.853622,Coma
4,table_1,website,companiesMarketCap_dataset.jsonl,website,0.853622,Coma
5,table_1,revenue,companiesMarketCap_dataset.jsonl,revenue,0.853622,Coma
6,table_1,name,companiesMarketCap_dataset.jsonl,name,0.844168,Coma
7,table_1,employees,companiesMarketCap_dataset.jsonl,number_of_employees,0.585239,Coma
8,table_1,sector,companiesMarketCap_dataset.jsonl,services,0.368328,Coma


In [32]:
original_columns=["name", "industry", "market_cap", "employees", "country", "headquarters", "address", "sector", "ceo", "founded", "founders", "share_price", "website", "stock", "area_served", "revenue"]
schema_mediato = pd.DataFrame(columns=original_columns)

df_gd = pd.read_json('datasets/output_globaldata.json')
df_gd.rename(columns={"number_of_employees":"employees"}, inplace=True)

df_disfold = pd.read_json('datasets/output_disfold.json')

df_vt = pd.read_json('datasets/gren-value.today.json')
df_vt.rename(columns={"annual_revenue_USD":"revenue", "annual_net_income_USD":"earnings", "employees_number":"employees", "headquarters_country": "headquarters", "market_capitalization_2022": "market_cap", "CEO":"ceo"}, inplace=True)

df_vt2 = pd.read_json('datasets/valueToday2.json')
df_vt2.rename(columns={"marketCap":"market_cap", "country": "country", "CEO":"ceo"}, inplace=True)

schema_mediato = schema_mediato.append(df_disfold[:500], ignore_index=True)
schema_mediato = schema_mediato.append(df_vt[:500], ignore_index=True)
schema_mediato = schema_mediato.append(df_vt2[:500], ignore_index=True)
schema_mediato = schema_mediato.append(df_gd[:500], ignore_index=True)

schema_mediato[original_columns].to_json('schema_mediato_auto.json', orient='records')

C:\Users\andre\AppData\Local\Temp\ipykernel_14896\198771844.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  schema_mediato = schema_mediato.append(df_disfold[:500], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_14896\198771844.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  schema_mediato = schema_mediato.append(df_vt[:500], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_14896\198771844.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  schema_mediato = schema_mediato.append(df_vt2[:500], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_14896\198771844.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us